# Synchronization of multiple AWG output channels
This script implements the basic example as described in the corresponding [blog post](https://www.zhinst.com/blogs/synchronizing-multiple-awg-channels).

Copyright (C) 2022 Zurich Instruments

This software may be modified and distributed under the terms of the MIT license. See the LICENSE file for details.

### Imports and connection to instrument

In [1]:
from zhinst.toolkit import Session, Sequence
import copy

In [2]:
dev_id = 'dev8000'  # Instrument serial number
host = 'localhost'  # Host server IP address

DRIVE_CORE = 0      # The AWG core that generate drive pulses
GATE_CORE = 1       # The AWG core that generate gate pulses

In [3]:
session = Session(host)                     # Create a Session (connection to data server)
device = session.connect_device(dev_id)     # Connect to instrument

awg_drive = device.awgs[DRIVE_CORE]
awg_gate = device.awgs[GATE_CORE]

### Basic channel configuration

In [4]:
# use transactional set to configure multiple settings
with device.set_transaction():
    device.system.awg.channelgrouping(0)    # work in non-grouped mode (individual AWG cores)

    device.sigouts["*"].on(True)    # Switch on the outputs
    device.sigouts["*"].range(1)    # Set output voltage range
    device.awgs["*"].single(True)   # Disable 'Rerun' of AWG sequencer
    device.oscs[0].freq(111e6)      # Modulation frequency for drive pulse
    device.sines[0].phaseshift(90)  # Sine generator phase for IQ modulation
    device.sines[1].phaseshift(0)   # Sine generator phase for IQ modulation

    awg_drive.outputs["*"].modulation.mode('mixer')    # Enable digital modulation for drive pulses
    awg_gate.outputs["*"].modulation.mode('off')       # Disable digital modulation for gate pulses

    # coefficients of gain matrix for ideal mixer
    awg_drive.outputs[0].gains[0](0.5)
    awg_drive.outputs[0].gains[1](0.0)
    awg_drive.outputs[1].gains[0](0.0)
    awg_drive.outputs[1].gains[1](0.5)


## Sequencer code for leader and follower AWG core

In [5]:
# Sequencer program for leader AWG core
seq_leader = Sequence()
seq_leader.constants['LEN_SECTION'] = 1024
seq_leader.constants['LEN_PULSE'] = 256

seq_leader.code = """
wave w = gauss(LEN_PULSE, LEN_PULSE/2, LEN_PULSE/8);

setDIO(1);  //Send trigger
wait(5);
setDIO(0);

waitDIOTrigger();   //Wait for trigger

//Waveform playback 
// initialization stage
playZero(LEN_SECTION);
// manipulation stage
playZero(LEN_SECTION-LEN_PULSE);
playWave(1,2,w);
// readout stage
playZero(LEN_SECTION);
"""

# Sequencer program for follower AWG core
# copy the constants from the leader
seq_follower = copy.copy(seq_leader)

seq_follower.code = """
wave w = ones(LEN_SECTION);

waitDIOTrigger();   //Wait for trigger

//Waveform playback
// initialization stage
playWave(0.4*w, 0.4*w);
// manipulation stage
playWave(1.0*w, 1.0*w);
// readout stage
playWave(0.7*w, 0.7*w);
"""

In [6]:
# upload sequencer programs to individual AWG cores
with device.set_transaction():
    awg_drive.load_sequencer_program(seq_leader)
    awg_gate.load_sequencer_program(seq_follower)

## Configure DIO

We configure the following settings in the DIO interface:
* mode "AWG Sequencer"
* enable driving of first bus
* polarity, index and slope of trigger

In [7]:
with device.set_transaction():
    # Settings in 'DIO' tab
    device.dios[0].drive(0b0001)                        # Enable driving first byte
    device.dios[0].output(0x00)                         # Reset DIO to clean state
    device.dios[0].mode('awg_sequencer_commands')       # Switch to "AWG Sequencer" mode
    
    # Settings in "AWG Sequencer" - "Trigger" sub-tab
    device.awgs["*"].dio.valid.polarity('high')
    device.awgs["*"].dio.valid.index(0)
    device.awgs["*"].dio.strobe.slope('off')

## Run the sequence
First run the follower AWG cores, then the leader AWG core

In [8]:
# With the set_transaction method, the dataserver ensures the right order of execution
# The follower(s) must be started before of the leader
with device.set_transaction():
    awg_gate.enable(True)
    awg_drive.enable(True)